In [3]:
import pandas as pd
import requests
from io import StringIO
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

def get_us_oecd_data(indicator, start_date='2015', end_date='2024', timeout=30):
    base_url = 'https://stats.oecd.org/SDMX-JSON/data'

    indicators = {
        'gdp': 'QNA/USA.B1_GE.LNBQRSA.Q',
        'inflation': 'PRICES_CPI/USA.CPALTT01.GY.M',
        'employment': 'STLABOUR/USA.LRHUTTTT.ST.M',
        'industrial_production': 'MEI/USA.PRMNTO01.IXOBSA.M',
        'consumer_confidence': 'MEI/USA.CSCICP03.AMPLM.M'
    }

    # Configure retry strategy
    retry_strategy = Retry(
        total=3,
        backoff_factor=1,
        status_forcelist=[429, 500, 502, 503, 504]
    )
    adapter = HTTPAdapter(max_retries=retry_strategy)
    session = requests.Session()
    session.mount("https://", adapter)

    headers = {
        'Accept': 'text/csv',
        'Accept-Encoding': 'gzip'
    }

    try:
        url = f'{base_url}/{indicators[indicator]}/all?startTime={start_date}&endTime={end_date}&dimensionAtObservation=AllDimensions'
        print(url)
        response = session.get(url, headers=headers, timeout=timeout)
        response.raise_for_status()
        return pd.read_csv(StringIO(response.text))
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None
    finally:
        session.close()


In [4]:
# Usage
gdp_data = get_us_oecd_data('gdp')

https://stats.oecd.org/SDMX-JSON/data/QNA/USA.B1_GE.LNBQRSA.Q/all?startTime=2015&endTime=2024&dimensionAtObservation=AllDimensions


KeyboardInterrupt: 